In [32]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from ast import literal_eval

In [124]:
movies = pd.read_csv('./data/tmdb_5000_movies.csv')
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


`genres`, `keywords` 기반 추천

In [125]:
movies = movies[['title', 'genres', 'keywords']]

`genres`, `keywords`가 dictionary가 아닌 str이기 때문에, str을 list of dictionaries로 변환시켜야 함

In [126]:
# str -> list of dicts
movies['genres'] = movies['genres'].apply(literal_eval) 
movies['keywords'] = movies['keywords'].apply(literal_eval) 
movies.head()

,title,genres,keywords
0,Avatar,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...","[{'id': 1463, 'name': 'culture clash'}, {'id':..."
1,Pirates of the Caribbean: At World's End,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'id': 270, 'name': 'ocean'}, {'id': 726, 'na..."
2,Spectre,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...","[{'id': 470, 'name': 'spy'}, {'id': 818, 'name..."
3,The Dark Knight Rises,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...","[{'id': 849, 'name': 'dc comics'}, {'id': 853,..."
4,John Carter,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...","[{'id': 818, 'name': 'based on novel'}, {'id':..."


In [127]:
# name만 추출
movies['genres'] = movies['genres'].apply(lambda x: [a['name'] for a in x])
movies['keywords'] = movies['keywords'].apply(lambda x: [a['name'] for a in x])
movies.head()

,title,genres,keywords
0,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon..."
1,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ..."
2,Spectre,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi..."
3,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i..."
4,John Carter,"[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel..."


데이터 정리가 잘 되어 있어서, 대소문자 문제 없음. 따라서 소문자로 변환하는 전처리할 필요없음.

In [128]:
# list -> str
movies['genres'] = movies['genres'].apply(lambda x: ' '.join(x))
movies['keywords'] = movies['keywords'].apply(lambda x: ' '.join(x))
movies.head()

,title,genres,keywords
0,Avatar,Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...
1,Pirates of the Caribbean: At World's End,Adventure Fantasy Action,ocean drug abuse exotic island east india trad...
2,Spectre,Action Adventure Crime,spy based on novel secret agent sequel mi6 bri...
3,The Dark Knight Rises,Action Crime Drama Thriller,dc comics crime fighter terrorist secret ident...
4,John Carter,Action Adventure Science Fiction,based on novel mars medallion space travel pri...


In [129]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vec = TfidfVectorizer(ngram_range=(1, 2)) # space war과 같은 단어들 때문에 n-gram을 2까지
tfidf_matrix = tfidf_vec.fit_transform(movies['genres'])
# print(tfidf_vec.vocabulary_.items())
print(tfidf_matrix.shape) # 장르(276), 키워드(41554)

(4803, 276)


In [130]:
from sklearn.metrics.pairwise import cosine_similarity
genres_similarity = cosine_similarity(tfidf_matrix, tfidf_matrix) # 각 행끼리의 유사도
print(genres_similarity) 
print(genres_similarity.shape)

[[1.         0.49309367 0.29270708 ... 0.         0.         0.        ]
 [0.49309367 1.         0.17786505 ... 0.         0.         0.        ]
 [0.29270708 0.17786505 1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]
(4803, 4803)


값이 존재하는 행의 경우에 대각 행렬 값 = 1 (0인 경우는 missing value)

In [131]:
# similarity가 높은 값만 추출
similar_index = np.argsort(-genres_similarity) # 큰 값부터 추출하기 위해 minus 처리함
print(similar_index)

[[   0  870 3494 ... 2355 2397 4802]
 [ 329  379  199 ... 2323 2325 4802]
 [1740    2 1542 ... 2359 2344 4802]
 ...
 [4800 3809 3285 ... 2085 2142 4802]
 [   0 3205 3204 ... 1596 1594 4802]
 [4802 4593 4583 ... 1597 1595 2401]]


In [132]:
# genres
input_movie = input()

movie_index = movies[movies['title'] == input_movie].index.values # 몇 번째 행에 있는 영화인지
print(movie_index)
similar_movies = similar_index[movie_index, :int(10)] # 상위 10개만 추출
print(similar_movies)
similar_movies_index = similar_movies.reshape(-1) # 1차원으로 변경
print(similar_movies_index)
print(movies.iloc[similar_movies_index])

Inception
[96]
[[  96 2966  275 2897 2484 4639 1506  908  564  279]]
[  96 2966  275 2897 2484 4639 1506  908  564  279]
                           title                                             genres                                           keywords
96                     Inception  Action Thriller Science Fiction Mystery Adventure  loss of lover dream kidnapping sleep subconsci...
2966       2001: A Space Odyssey                  Science Fiction Mystery Adventure  moon jupiter artificial intelligence man vs ma...
275              Minority Report            Action Thriller Science Fiction Mystery  self-fulfilling prophecy washington d.c. evide...
2897                      Cypher                   Thriller Science Fiction Mystery  double life undercover lie wife company indust...
2484        The Thirteenth Floor                   Thriller Science Fiction Mystery  artificial intelligence simulation computer pr...
4639                        Cube                   Thriller Science F

In [123]:
# keywords
input_movie = input()

movie_index = movies[movies['title'] == input_movie].index.values # 몇 번째 행에 있는 영화인지
print(movie_index)
similar_movies = similar_index[movie_index, :int(10)] # 상위 10개만 추출
print(similar_movies)
similar_movies_index = similar_movies.reshape(-1) # 1차원으로 변경
print(similar_movies_index)
print(movies.iloc[similar_movies_index])

Inception
[96]
[[  96  448 1193 1503 2592 2000 4590 2682 1668 1136]]
[  96  448 1193 1503 2592 2000 4590 2682 1668 1136]
                          title                                             genres                                           keywords
96                    Inception  Action Thriller Science Fiction Mystery Adventure  loss of lover dream kidnapping sleep subconsci...
448               Cold Mountain                                              Drama  loss of lover loss of family deserter loss of ...
1193  The Count of Monte Cristo                    Action Adventure Drama Thriller  loss of lover lover (female) ex-lover torture ...
1503                     Takers                        Action Crime Drama Thriller                                              heist
2592        Highlander: Endgame                     Action Fantasy Science Fiction  loss of lover antiquary loss of powers death o...
2000          Anywhere But Here                               Drama Comedy 

In [ ]:
# tfidf와 다른 벡터화
from sklearn.feature_extraction.text import CountVectorizer

count_vec = CountVectorizer(ngram_range=(1,2))
count_matrix = count_vec.fit_transform(movies['keywords'])

In [93]:
print(tfidf_matrix.shape)

(4803, 276)


# 장르, 키워드 둘 다 사용

### 방법1) `genres` + `keywords` 합친 `total` column을 사용 -> `genres`를 이용한 경우와 똑같이 나옴

In [150]:
movies = pd.read_csv('./data/tmdb_5000_movies.csv')
movies = movies[['title', 'genres', 'keywords']]

movies['genres'] = movies['genres'].apply(literal_eval) 
movies['keywords'] = movies['keywords'].apply(literal_eval) 
movies['genres'] = movies['genres'].apply(lambda x: [a['name'] for a in x])
movies['keywords'] = movies['keywords'].apply(lambda x: [a['name'] for a in x])
movies['genres'] = movies['genres'].apply(lambda x: ' '.join(x))
movies['keywords'] = movies['keywords'].apply(lambda x: ' '.join(x))

# 추가
movies['total'] = movies['genres'] + ' ' + movies['keywords']

In [151]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tfidf_vec = TfidfVectorizer(ngram_range=(1, 2)) 
tfidf_matrix = tfidf_vec.fit_transform(movies['total'])
print(tfidf_matrix.shape)

similarity_total = cosine_similarity(tfidf_matrix, tfidf_matrix)
similarity_index = np.argsort(-similarity_total) # 44261개
print(similar_index)

(4803, 44261)
[[   0  870 3494 ... 2355 2397 4802]
 [ 329  379  199 ... 2323 2325 4802]
 [1740    2 1542 ... 2359 2344 4802]
 ...
 [4800 3809 3285 ... 2085 2142 4802]
 [   0 3205 3204 ... 1596 1594 4802]
 [4802 4593 4583 ... 1597 1595 2401]]


In [153]:
# total1
input_movie = input()

movie_index = movies[movies['title'] == input_movie].index.values 
similar_movies = similar_index[movie_index, :int(10)] 
similar_movies_index = similar_movies.reshape(-1) 
print(movies.iloc[similar_movies_index]['title'])

Mad Max
4670                        Mad Max
122        X-Men Origins: Wolverine
300               Starship Troopers
4077    Mad Max 2: The Road Warrior
587                       The Abyss
125             The Matrix Reloaded
76      G.I. Joe: The Rise of Cobra
123          The Matrix Revolutions
41                    Green Lantern
334               Jurassic Park III
Name: title, dtype: object


### 방법2) `genres`로 만든 matrix와 `keywords`로 만든 matrix를 결합해서 사용

In [133]:
movies = pd.read_csv('./data/tmdb_5000_movies.csv')
movies = movies[['title', 'genres', 'keywords']]

movies['genres'] = movies['genres'].apply(literal_eval) 
movies['keywords'] = movies['keywords'].apply(literal_eval) 
movies['genres'] = movies['genres'].apply(lambda x: [a['name'] for a in x])
movies['keywords'] = movies['keywords'].apply(lambda x: [a['name'] for a in x])
movies['genres'] = movies['genres'].apply(lambda x: ' '.join(x))
movies['keywords'] = movies['keywords'].apply(lambda x: ' '.join(x))

In [147]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import scipy.sparse

tfidf_vec_g = TfidfVectorizer(ngram_range=(1, 2)) 
tfidf_matrix_g = tfidf_vec_g.fit_transform(movies['genres'])
print(tfidf_matrix_g.shape)

tfidf_vec_k = TfidfVectorizer(ngram_range=(1, 2)) 
tfidf_matrix_k = tfidf_vec_k.fit_transform(movies['keywords'])
print(tfidf_matrix_k.shape)

tfidf_matrix = np.concatenate([tfidf_matrix_g.toarray(), tfidf_matrix_k.toarray()], axis=1)
tfidf_matrix = scipy.sparse.csr_matrix(tfidf_matrix)
print(tfidf_matrix.shape)

similarity_total = cosine_similarity(tfidf_matrix, tfidf_matrix)
similarity_index = np.argsort(-similarity_total) 
print(similar_index)

(4803, 276)
(4803, 41554)
(4803, 41830)
[[   0  870 3494 ... 2355 2397 4802]
 [ 329  379  199 ... 2323 2325 4802]
 [1740    2 1542 ... 2359 2344 4802]
 ...
 [4800 3809 3285 ... 2085 2142 4802]
 [   0 3205 3204 ... 1596 1594 4802]
 [4802 4593 4583 ... 1597 1595 2401]]


In [148]:
# total2
input_movie = input()

movie_index = movies[movies['title'] == input_movie].index.values 
similar_movies = similar_index[movie_index, :int(10)] 
similar_movies_index = similar_movies.reshape(-1) 
print(movies.iloc[similar_movies_index]['title'])

Inception
96                       Inception
2966         2001: A Space Odyssey
275                Minority Report
2897                        Cypher
2484          The Thirteenth Floor
4639                          Cube
1506                   Source Code
908                        Super 8
564                          Signs
279     Terminator 2: Judgment Day
Name: title, dtype: object
